In [1]:
# import the libraries
import numpy as np
import pandas as pd
import re 
from Bio import SeqIO
from collections import defaultdict

In [2]:
# Create a dataframe for Bandage Combined1
master_df= pd.read_csv('QueriesAndResultsMasterList.tsv', sep='\t')

In [3]:
master_df

,Unnamed: 0,Query,Sequence_Query,Length_Bandage,Path_Bandage,Start_Bandage,End_Bandage,Extracted_Path,Sequence_Bandage,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner,Path_GraphAligner,Start_GraphAligner,End_GraphAligner,BandageVSSPAligner,SPAlignerVSGraphAligner
0,0,gb|AB113580|+|1329-2193|ARO:3002332|GES-3,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,864,"(93) 32422725-, 5613425+ (32) , (93) 32422725-...","93, 93","32, 32","32422725, 5613425,32422725, 32146774",ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,94,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,32422725,93,956,False,False
1,1,gb|AB114632|+|655-1453|ARO:3002607|aadA7,ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,798,"(61) 26862905- (858) , (22) 3906318-","61, 22","858, 3906318","26862905,3906318",ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,62,803,743,798,ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,26862905,61,858,False,False
2,2,gb|AB116260|+|1329-2193|ARO:3002333|GES-4,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,864,"(93) 32422725-, 5613425+ (32) , (93) 32422725-...","93, 93","32, 32","32422725, 5613425,32422725, 32146774",ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,94,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,32422725,93,956,False,False
3,3,gb|AB211124|+|0-525|ARO:3002895|SAT-1,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...,525,(64) 81232484- (588),64,588,81232484,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...,65,533,470,525,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...,81232484,64,588,False,False
4,4,gb|AB571865.1|-|143423-144308|ARO:3003742|mphG,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,885,"(353) 50056+, 2638631+, 6283287+, 5689755- (17...","353, 353, 171, 171, 171, 171","176, 176, 176, 176, 176, 176","50056, 2638631, 6283287, 5689755,50056, 263863...",ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,354,884,532,885,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,"50056, 2638631, 6283287, 5689755",353,1237,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,136,gb|X07753|+|102-882|ARO:3002652|APH(3')-VIa,ATGGAATTGCCCAATATTATTCAACAATTTATCGGAAACAGCGTTT...,780,"(180) 6903442+, 506744- (243) , (180) 6903442+...","180, 180","243, 241","6903442, 506744,6903442, 86408359",ATGGAATTGCCTAATATTATTCAACAATTTATTGGAAACAGCGTTT...,181,716,537,780,ATGGAATTGCCTAATATTATTCAACAATTTATTGGAAACAGCGTTT...,"6903442, 506744",180,959,False,False
137,137,gb|X13543.1|+|185-1046|ARO:3002533|AAC(3)-IIa,ATGCATACGCAGAAGGCAATAACGGAGGCGCTTCAAAAACTCGGAG...,861,(130) 605844+ (990),130,990,605844,ATGCATACGCGGAAGGCAATAACGGAGGCAATTCGAAAACTCGGAG...,131,935,806,861,ATGCATACGCGGAAGGCAATAACGGAGGCAATTCGAAAACTCGGAG...,605844,130,990,False,False
138,138,gb|X75562|+|134-935|ARO:3001402|OXA-7,ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,801,"(72) 32592258-, 758212-, 74766869- (55) , (72)...","72, 72, 72, 72, 23, 23","55, 55, 55, 55, 55, 55","32592258, 758212, 74766869,32592258, 758212, 3...",ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,2,626,626,801,TTCGTGCTTTGTAAAAGTAGCAGTAAATCCTGCGCTACCAATGACT...,"32592258, 758212",72,872,False,False
139,139,gb|Z21523|+|0-1974|ARO:3000191|tetQ,GTGCGTTTCGACAATGCATCTATTGTAGTATATTATTGCTTAATCC...,1974,"(142) 4764923-, 2707722+, 11583809+, 43494081-...","142, 142, 1106, 1106, 142, 1106, 142, 68, 55, ...","68, 55, 68, 55, 80, 80, 43494077, 68, 55, 4349...","4764923, 2707722, 11583809, 43494081, 43494077...",GTGCGTTTCGACAATGCATCTACTGTAGTATATTATTGCTTAATCC...,2,1708,1708,1974,ATTATAAATTTAGGAATTCTTGCTCACATTGATGCAGGAAAAACTT...,"4764923, 2707722, 11583809, 43494081, 43494077...",142,2115,False,False


In [4]:
#Drop the columns
master_df=master_df.drop(['BandageVSSPAligner','SPAlignerVSGraphAligner'],axis=1)

In [5]:
master_df

,Unnamed: 0,Query,Sequence_Query,Length_Bandage,Path_Bandage,Start_Bandage,End_Bandage,Extracted_Path,Sequence_Bandage,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner,Path_GraphAligner,Start_GraphAligner,End_GraphAligner
0,0,gb|AB113580|+|1329-2193|ARO:3002332|GES-3,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,864,"(93) 32422725-, 5613425+ (32) , (93) 32422725-...","93, 93","32, 32","32422725, 5613425,32422725, 32146774",ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,94,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,32422725,93,956
1,1,gb|AB114632|+|655-1453|ARO:3002607|aadA7,ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,798,"(61) 26862905- (858) , (22) 3906318-","61, 22","858, 3906318","26862905,3906318",ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,62,803,743,798,ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,26862905,61,858
2,2,gb|AB116260|+|1329-2193|ARO:3002333|GES-4,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,864,"(93) 32422725-, 5613425+ (32) , (93) 32422725-...","93, 93","32, 32","32422725, 5613425,32422725, 32146774",ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,94,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,32422725,93,956
3,3,gb|AB211124|+|0-525|ARO:3002895|SAT-1,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...,525,(64) 81232484- (588),64,588,81232484,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...,65,533,470,525,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...,81232484,64,588
4,4,gb|AB571865.1|-|143423-144308|ARO:3003742|mphG,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,885,"(353) 50056+, 2638631+, 6283287+, 5689755- (17...","353, 353, 171, 171, 171, 171","176, 176, 176, 176, 176, 176","50056, 2638631, 6283287, 5689755,50056, 263863...",ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,354,884,532,885,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,"50056, 2638631, 6283287, 5689755",353,1237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,136,gb|X07753|+|102-882|ARO:3002652|APH(3')-VIa,ATGGAATTGCCCAATATTATTCAACAATTTATCGGAAACAGCGTTT...,780,"(180) 6903442+, 506744- (243) , (180) 6903442+...","180, 180","243, 241","6903442, 506744,6903442, 86408359",ATGGAATTGCCTAATATTATTCAACAATTTATTGGAAACAGCGTTT...,181,716,537,780,ATGGAATTGCCTAATATTATTCAACAATTTATTGGAAACAGCGTTT...,"6903442, 506744",180,959
137,137,gb|X13543.1|+|185-1046|ARO:3002533|AAC(3)-IIa,ATGCATACGCAGAAGGCAATAACGGAGGCGCTTCAAAAACTCGGAG...,861,(130) 605844+ (990),130,990,605844,ATGCATACGCGGAAGGCAATAACGGAGGCAATTCGAAAACTCGGAG...,131,935,806,861,ATGCATACGCGGAAGGCAATAACGGAGGCAATTCGAAAACTCGGAG...,605844,130,990
138,138,gb|X75562|+|134-935|ARO:3001402|OXA-7,ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,801,"(72) 32592258-, 758212-, 74766869- (55) , (72)...","72, 72, 72, 72, 23, 23","55, 55, 55, 55, 55, 55","32592258, 758212, 74766869,32592258, 758212, 3...",ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,2,626,626,801,TTCGTGCTTTGTAAAAGTAGCAGTAAATCCTGCGCTACCAATGACT...,"32592258, 758212",72,872
139,139,gb|Z21523|+|0-1974|ARO:3000191|tetQ,GTGCGTTTCGACAATGCATCTATTGTAGTATATTATTGCTTAATCC...,1974,"(142) 4764923-, 2707722+, 11583809+, 43494081-...","142, 142, 1106, 1106, 142, 1106, 142, 68, 55, ...","68, 55, 68, 55, 80, 80, 43494077, 68, 55, 4349...","4764923, 2707722, 11583809, 43494081, 43494077...",GTGCGTTTCGACAATGCATCTACTGTAGTATATTATTGCTTAATCC...,2,1708,1708,1974,ATTATAAATTTAGGAATTCTTGCTCACATTGATGCAGGAAAAACTT...,"4764923, 2707722, 11583809, 43494081, 43494077...",142,2115


In [6]:
#Remove the + or - from the Path_SPAligner
#master_df['Path_SPAligner']=master_df['Path_SPAligner'].str.replace(r'[+-]', '', regex=True)
master_df['End_Bandage']=master_df['End_Bandage'].str.replace(r'[+-]', '', regex=True)

In [7]:
#Add +2 to the values of Start_SPAligner to comapre with Start_GraphAligner
master_df['Start_SPAligner_GraphAligner'] = master_df['Start_SPAligner'].apply(lambda x: ', '.join(str(int(val) + 2) for val in x.split(',')))

AttributeError: 'int' object has no attribute 'split'

In [8]:
#Add +1 to the values of Start_SPAligner to comapre with Start_GraphAligner
master_df['Start_SPAligner_Bandage'] = master_df['Start_SPAligner'].apply(lambda x: ', '.join(str(int(val) + 1) for val in x.split(',')))

In [8]:
master_df

,Unnamed: 0,Query,Sequence_Query,Length_Bandage,Path_Bandage,Start_Bandage,End_Bandage,Extracted_Path,Sequence_Bandage,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner,Path_GraphAligner,Start_GraphAligner,End_GraphAligner
0,0,gb|AB113580|+|1329-2193|ARO:3002332|GES-3,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,864,"(93) 32422725-, 5613425+ (32) , (93) 32422725-...","93, 93","32, 32","32422725, 5613425,32422725, 32146774",ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,94,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,32422725,93,956
1,1,gb|AB114632|+|655-1453|ARO:3002607|aadA7,ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,798,"(61) 26862905- (858) , (22) 3906318-","61, 22","858, 3906318","26862905,3906318",ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,62,803,743,798,ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,26862905,61,858
2,2,gb|AB116260|+|1329-2193|ARO:3002333|GES-4,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,864,"(93) 32422725-, 5613425+ (32) , (93) 32422725-...","93, 93","32, 32","32422725, 5613425,32422725, 32146774",ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,94,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,32422725,93,956
3,3,gb|AB211124|+|0-525|ARO:3002895|SAT-1,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...,525,(64) 81232484- (588),64,588,81232484,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...,65,533,470,525,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...,81232484,64,588
4,4,gb|AB571865.1|-|143423-144308|ARO:3003742|mphG,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,885,"(353) 50056+, 2638631+, 6283287+, 5689755- (17...","353, 353, 171, 171, 171, 171","176, 176, 176, 176, 176, 176","50056, 2638631, 6283287, 5689755,50056, 263863...",ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,354,884,532,885,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,"50056, 2638631, 6283287, 5689755",353,1237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,136,gb|X07753|+|102-882|ARO:3002652|APH(3')-VIa,ATGGAATTGCCCAATATTATTCAACAATTTATCGGAAACAGCGTTT...,780,"(180) 6903442+, 506744- (243) , (180) 6903442+...","180, 180","243, 241","6903442, 506744,6903442, 86408359",ATGGAATTGCCTAATATTATTCAACAATTTATTGGAAACAGCGTTT...,181,716,537,780,ATGGAATTGCCTAATATTATTCAACAATTTATTGGAAACAGCGTTT...,"6903442, 506744",180,959
137,137,gb|X13543.1|+|185-1046|ARO:3002533|AAC(3)-IIa,ATGCATACGCAGAAGGCAATAACGGAGGCGCTTCAAAAACTCGGAG...,861,(130) 605844+ (990),130,990,605844,ATGCATACGCGGAAGGCAATAACGGAGGCAATTCGAAAACTCGGAG...,131,935,806,861,ATGCATACGCGGAAGGCAATAACGGAGGCAATTCGAAAACTCGGAG...,605844,130,990
138,138,gb|X75562|+|134-935|ARO:3001402|OXA-7,ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,801,"(72) 32592258-, 758212-, 74766869- (55) , (72)...","72, 72, 72, 72, 23, 23","55, 55, 55, 55, 55, 55","32592258, 758212, 74766869,32592258, 758212, 3...",ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,2,626,626,801,TTCGTGCTTTGTAAAAGTAGCAGTAAATCCTGCGCTACCAATGACT...,"32592258, 758212",72,872
139,139,gb|Z21523|+|0-1974|ARO:3000191|tetQ,GTGCGTTTCGACAATGCATCTATTGTAGTATATTATTGCTTAATCC...,1974,"(142) 4764923-, 2707722+, 11583809+, 43494081-...","142, 142, 1106, 1106, 142, 1106, 142, 68, 55, ...","68, 55, 68, 55, 80, 80, 43494077, 68, 55, 4349...","4764923, 2707722, 11583809, 43494081, 43494077...",GTGCGTTTCGACAATGCATCTACTGTAGTATATTATTGCTTAATCC...,2,1708,1708,1974,ATTATAAATTTAGGAATTCTTGCTCACATTGATGCAGGAAAAACTT...,"4764923, 2707722, 11583809, 43494081, 43494077...",142,2115


In [9]:
#Remove the braces from the end_Bandage
master_df['End_Bandage'] = master_df['End_Bandage'].str.replace('(', '').str.replace(')', '')
master_df

,Unnamed: 0,Query,Sequence_Query,Length_Bandage,Path_Bandage,Start_Bandage,End_Bandage,Extracted_Path,Sequence_Bandage,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner,Path_GraphAligner,Start_GraphAligner,End_GraphAligner
0,0,gb|AB113580|+|1329-2193|ARO:3002332|GES-3,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,864,"(93) 32422725-, 5613425+ (32) , (93) 32422725-...","93, 93","32, 32","32422725, 5613425,32422725, 32146774",ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,94,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,32422725,93,956
1,1,gb|AB114632|+|655-1453|ARO:3002607|aadA7,ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,798,"(61) 26862905- (858) , (22) 3906318-","61, 22","858, 3906318","26862905,3906318",ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,62,803,743,798,ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,26862905,61,858
2,2,gb|AB116260|+|1329-2193|ARO:3002333|GES-4,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,864,"(93) 32422725-, 5613425+ (32) , (93) 32422725-...","93, 93","32, 32","32422725, 5613425,32422725, 32146774",ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,94,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,32422725,93,956
3,3,gb|AB211124|+|0-525|ARO:3002895|SAT-1,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...,525,(64) 81232484- (588),64,588,81232484,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...,65,533,470,525,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...,81232484,64,588
4,4,gb|AB571865.1|-|143423-144308|ARO:3003742|mphG,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,885,"(353) 50056+, 2638631+, 6283287+, 5689755- (17...","353, 353, 171, 171, 171, 171","176, 176, 176, 176, 176, 176","50056, 2638631, 6283287, 5689755,50056, 263863...",ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,354,884,532,885,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,"50056, 2638631, 6283287, 5689755",353,1237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,136,gb|X07753|+|102-882|ARO:3002652|APH(3')-VIa,ATGGAATTGCCCAATATTATTCAACAATTTATCGGAAACAGCGTTT...,780,"(180) 6903442+, 506744- (243) , (180) 6903442+...","180, 180","243, 241","6903442, 506744,6903442, 86408359",ATGGAATTGCCTAATATTATTCAACAATTTATTGGAAACAGCGTTT...,181,716,537,780,ATGGAATTGCCTAATATTATTCAACAATTTATTGGAAACAGCGTTT...,"6903442, 506744",180,959
137,137,gb|X13543.1|+|185-1046|ARO:3002533|AAC(3)-IIa,ATGCATACGCAGAAGGCAATAACGGAGGCGCTTCAAAAACTCGGAG...,861,(130) 605844+ (990),130,990,605844,ATGCATACGCGGAAGGCAATAACGGAGGCAATTCGAAAACTCGGAG...,131,935,806,861,ATGCATACGCGGAAGGCAATAACGGAGGCAATTCGAAAACTCGGAG...,605844,130,990
138,138,gb|X75562|+|134-935|ARO:3001402|OXA-7,ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,801,"(72) 32592258-, 758212-, 74766869- (55) , (72)...","72, 72, 72, 72, 23, 23","55, 55, 55, 55, 55, 55","32592258, 758212, 74766869,32592258, 758212, 3...",ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,2,626,626,801,TTCGTGCTTTGTAAAAGTAGCAGTAAATCCTGCGCTACCAATGACT...,"32592258, 758212",72,872
139,139,gb|Z21523|+|0-1974|ARO:3000191|tetQ,GTGCGTTTCGACAATGCATCTATTGTAGTATATTATTGCTTAATCC...,1974,"(142) 4764923-, 2707722+, 11583809+, 43494081-...","142, 142, 1106, 1106, 142, 1106, 142, 68, 55, ...","68, 55, 68, 55, 80, 80, 43494077, 68, 55, 4349...","4764923, 2707722, 11583809, 43494081, 43494077...",GTGCGTTTCGACAATGCATCTACTGTAGTATATTATTGCTTAATCC...,2,1708,1708,1974,ATTATAAATTTAGGAATTCTTGCTCACATTGATGCAGGAAAAACTT...,"4764923, 2707722, 11583809, 43494081, 43494077...",142,2115


## This section compares the result from SPAligner and Bandage

In [10]:
#Convert Path_GraphAligner, Start_GraphAligner and End_GraphAligner into int
master_df= master_df.fillna(000)
master_df['Start_GraphAligner']=master_df['Start_GraphAligner'].astype(int)
master_df['End_GraphAligner']=master_df['End_GraphAligner'].astype(int)
#master_df['Path_GraphAligner']=master_df['Path_GraphAligner'].astype(int)
master_df

ValueError: invalid literal for int() with base 10: '13, 13'

In [11]:
master_df.dtypes

Unnamed: 0             int64
Query                 object
Sequence_Query        object
Length_Bandage         int64
Path_Bandage          object
Start_Bandage         object
End_Bandage           object
Extracted_Path        object
Sequence_Bandage      object
Start_SPAligner        int64
End_SPAligner          int64
Length_SPAligner      object
Path_SPAligner         int64
Sequence_SPAligner    object
Path_GraphAligner     object
Start_GraphAligner    object
End_GraphAligner      object
dtype: object

In [13]:
master_df['Start_SPAligner']=master_df['Start_SPAligner'].astype(str)
#master_df['Start_SPAligner_GraphAligner']=master_df['Start_SPAligner_GraphAligner'].astype(str)
#master_df['Start_SPAligner_Bandage']=master_df['Start_SPAligner_Bandage'].astype(str)

master_df['End_SPAligner']=master_df['End_SPAligner'].astype(str)
master_df['Path_SPAligner']=master_df['Path_SPAligner'].astype(str)

In [14]:
master_df

,Unnamed: 0,Query,Sequence_Query,Length_Bandage,Path_Bandage,Start_Bandage,End_Bandage,Extracted_Path,Sequence_Bandage,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner,Path_GraphAligner,Start_GraphAligner,End_GraphAligner
0,0,gb|AB113580|+|1329-2193|ARO:3002332|GES-3,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,864,"(93) 32422725-, 5613425+ (32) , (93) 32422725-...","93, 93","32, 32","32422725, 5613425,32422725, 32146774",ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,94,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,32422725,93,956
1,1,gb|AB114632|+|655-1453|ARO:3002607|aadA7,ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,798,"(61) 26862905- (858) , (22) 3906318-","61, 22","858, 3906318","26862905,3906318",ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,62,803,743,798,ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,26862905,61,858
2,2,gb|AB116260|+|1329-2193|ARO:3002333|GES-4,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,864,"(93) 32422725-, 5613425+ (32) , (93) 32422725-...","93, 93","32, 32","32422725, 5613425,32422725, 32146774",ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,94,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,32422725,93,956
3,3,gb|AB211124|+|0-525|ARO:3002895|SAT-1,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...,525,(64) 81232484- (588),64,588,81232484,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...,65,533,470,525,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...,81232484,64,588
4,4,gb|AB571865.1|-|143423-144308|ARO:3003742|mphG,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,885,"(353) 50056+, 2638631+, 6283287+, 5689755- (17...","353, 353, 171, 171, 171, 171","176, 176, 176, 176, 176, 176","50056, 2638631, 6283287, 5689755,50056, 263863...",ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,354,884,532,885,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,"50056, 2638631, 6283287, 5689755",353,1237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,136,gb|X07753|+|102-882|ARO:3002652|APH(3')-VIa,ATGGAATTGCCCAATATTATTCAACAATTTATCGGAAACAGCGTTT...,780,"(180) 6903442+, 506744- (243) , (180) 6903442+...","180, 180","243, 241","6903442, 506744,6903442, 86408359",ATGGAATTGCCTAATATTATTCAACAATTTATTGGAAACAGCGTTT...,181,716,537,780,ATGGAATTGCCTAATATTATTCAACAATTTATTGGAAACAGCGTTT...,"6903442, 506744",180,959
137,137,gb|X13543.1|+|185-1046|ARO:3002533|AAC(3)-IIa,ATGCATACGCAGAAGGCAATAACGGAGGCGCTTCAAAAACTCGGAG...,861,(130) 605844+ (990),130,990,605844,ATGCATACGCGGAAGGCAATAACGGAGGCAATTCGAAAACTCGGAG...,131,935,806,861,ATGCATACGCGGAAGGCAATAACGGAGGCAATTCGAAAACTCGGAG...,605844,130,990
138,138,gb|X75562|+|134-935|ARO:3001402|OXA-7,ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,801,"(72) 32592258-, 758212-, 74766869- (55) , (72)...","72, 72, 72, 72, 23, 23","55, 55, 55, 55, 55, 55","32592258, 758212, 74766869,32592258, 758212, 3...",ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,2,626,626,801,TTCGTGCTTTGTAAAAGTAGCAGTAAATCCTGCGCTACCAATGACT...,"32592258, 758212",72,872
139,139,gb|Z21523|+|0-1974|ARO:3000191|tetQ,GTGCGTTTCGACAATGCATCTATTGTAGTATATTATTGCTTAATCC...,1974,"(142) 4764923-, 2707722+, 11583809+, 43494081-...","142, 142, 1106, 1106, 142, 1106, 142, 68, 55, ...","68, 55, 68, 55, 80, 80, 43494077, 68, 55, 4349...","4764923, 2707722, 11583809, 43494081, 43494077...",GTGCGTTTCGACAATGCATCTACTGTAGTATATTATTGCTTAATCC...,2,1708,1708,1974,ATTATAAATTTAGGAATTCTTGCTCACATTGATGCAGGAAAAACTT...,"4764923, 2707722, 11583809, 43494081, 43494077...",142,2115


In [18]:
def compare_values_Bandage_SPAligner(row):
    if row['Start_Bandage'] == row['Start_SPAligner'] and row['End_Bandage'] == row['End_SPAligner'] and row['Extracted_Path'] == row['Path_SPAligner']:
        return 'Full'
    elif row['Extracted_Path'] == row['Path_SPAligner']and row['Start_Bandage'] != row['Start_SPAligner'] and row['End_Bandage'] != row['End_SPAligner']:
        return 'MatchOnPath'
    elif row['Extracted_Path'] == row['Path_SPAligner'] and row['Start_Bandage'] == row['Start_SPAligner'] and row['End_Bandage'] != row['End_SPAligner'] :
        return 'MatchOnPathAndStart'
    elif row['Extracted_Path'] == row['Path_SPAligner'] and row['Start_Bandage'] != row['Start_SPAligner'] and row['End_Bandage'] == row['End_SPAligner'] :
        return 'MatchOnPathAndEnd'
    else:
        return 'Different'

In [19]:
master_df['BandageVSSPAligner'] = master_df.apply(compare_values_Bandage_SPAligner, axis=1)

In [17]:
master_df.dtypes

Unnamed: 0             int64
Query                 object
Sequence_Query        object
Length_Bandage         int64
Path_Bandage          object
Start_Bandage         object
End_Bandage           object
Extracted_Path        object
Sequence_Bandage      object
Start_SPAligner       object
End_SPAligner         object
Length_SPAligner      object
Path_SPAligner        object
Sequence_SPAligner    object
Path_GraphAligner     object
Start_GraphAligner    object
End_GraphAligner      object
dtype: object

In [18]:
master_df.to_csv("TestCompare1.tsv",sep='\t')

In [20]:
BandagevsSPAligner_df=master_df[['Query','Path_Bandage','Start_Bandage','End_Bandage','Extracted_Path',
                                'Start_SPAligner','End_SPAligner','Path_SPAligner']]

In [21]:
BandagevsSPAligner_df

,Query,Path_Bandage,Start_Bandage,End_Bandage,Extracted_Path,Start_SPAligner,End_SPAligner,Path_SPAligner
0,gb|AB113580|+|1329-2193|ARO:3002332|GES-3,"(93) 32422725-, 5613425+ (32) , (93) 32422725-...","93, 93","32, 32","32422725, 5613425,32422725, 32146774",94,901,864
1,gb|AB114632|+|655-1453|ARO:3002607|aadA7,"(61) 26862905- (858) , (22) 3906318-","61, 22","858, 3906318","26862905,3906318",62,803,798
2,gb|AB116260|+|1329-2193|ARO:3002333|GES-4,"(93) 32422725-, 5613425+ (32) , (93) 32422725-...","93, 93","32, 32","32422725, 5613425,32422725, 32146774",94,901,864
3,gb|AB211124|+|0-525|ARO:3002895|SAT-1,(64) 81232484- (588),64,588,81232484,65,533,525
4,gb|AB571865.1|-|143423-144308|ARO:3003742|mphG,"(353) 50056+, 2638631+, 6283287+, 5689755- (17...","353, 353, 171, 171, 171, 171","176, 176, 176, 176, 176, 176","50056, 2638631, 6283287, 5689755,50056, 263863...",354,884,885
...,...,...,...,...,...,...,...,...
136,gb|X07753|+|102-882|ARO:3002652|APH(3')-VIa,"(180) 6903442+, 506744- (243) , (180) 6903442+...","180, 180","243, 241","6903442, 506744,6903442, 86408359",181,716,780
137,gb|X13543.1|+|185-1046|ARO:3002533|AAC(3)-IIa,(130) 605844+ (990),130,990,605844,131,935,861
138,gb|X75562|+|134-935|ARO:3001402|OXA-7,"(72) 32592258-, 758212-, 74766869- (55) , (72)...","72, 72, 72, 72, 23, 23","55, 55, 55, 55, 55, 55","32592258, 758212, 74766869,32592258, 758212, 3...",2,626,801
139,gb|Z21523|+|0-1974|ARO:3000191|tetQ,"(142) 4764923-, 2707722+, 11583809+, 43494081-...","142, 142, 1106, 1106, 142, 1106, 142, 68, 55, ...","68, 55, 68, 55, 80, 80, 43494077, 68, 55, 4349...","4764923, 2707722, 11583809, 43494081, 43494077...",2,1708,1974


In [22]:
BandagevsSPAligner_df['BandageVSSPAligner'] = BandagevsSPAligner_df.apply(compare_values_Bandage_SPAligner, axis=1)

/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_2076/1140914095.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BandagevsSPAligner_df['BandageVSSPAligner'] = BandagevsSPAligner_df.apply(compare_values_Bandage_SPAligner, axis=1)


In [23]:
BandagevsSPAligner_df.to_csv('TestRes.tsv',sep='\t')

In [24]:
BandagevsSPAligner_df.dtypes

Query                 object
Path_Bandage          object
Start_Bandage         object
End_Bandage           object
Extracted_Path        object
Start_SPAligner       object
End_SPAligner         object
Path_SPAligner        object
BandageVSSPAligner    object
dtype: object

In [26]:
output_list = []

for index, row in BandagevsSPAligner_df.iterrows():
    path_values = str(row['Extracted_Path']).split(',')
    start_values = str(row['Start_Bandage']).split(',')
    end_values = str(row['End_Bandage']).split(',')
    
    path_value = row['Path_SPAligner']
    start_value= str(row['Start_SPAligner']).split(',')
    end_value= str(row['End_SPAligner']).split(',')
    
    if path_value in path_values  and start_value in start_values and end_value in end_values:
        output_list.append('SinglePathFull')
    elif path_value in path_values and start_value in start_values and end_value not in end_values:
        output_list.append('SinglePathStartMatch')
    elif path_value in path_values and start_value not in start_values and end_value in end_values:
        output_list.append('SinglePathEndMatch')
    elif path_value in path_values and start_value not in start_values and end_value not in end_values:
        output_list.append('SinglePathMatch')
    elif path_value not in path_values and start_value in start_values and end_value not in end_values:
        output_list.append('SingleStartMatch')
    else:
         output_list.append('Different')

BandagevsSPAligner_df['ResultsBandageVSSPAligner'] = output_list

/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_2076/3651665796.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BandagevsSPAligner_df['ResultsBandageVSSPAligner'] = output_list


In [27]:
def determine_final_result(row):
    if row['BandageVSSPAligner'] == 'Full':
        return 'Full'
    elif row['BandageVSSPAligner'] == 'MatchOnPath':
        return 'MatchOnPath'
    else:
        return row['ResultsBandageVSSPAligner']

In [28]:
BandagevsSPAligner_df.dtypes

Query                        object
Path_Bandage                 object
Start_Bandage                object
End_Bandage                  object
Extracted_Path               object
Start_SPAligner              object
End_SPAligner                object
Path_SPAligner               object
BandageVSSPAligner           object
ResultsBandageVSSPAligner    object
dtype: object

In [29]:
BandagevsSPAligner_df['FinalResultBandageVSSPAligner'] = BandagevsSPAligner_df.apply(determine_final_result, axis=1)

/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_2076/761307203.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BandagevsSPAligner_df['FinalResultBandageVSSPAligner'] = BandagevsSPAligner_df.apply(determine_final_result, axis=1)


In [30]:
BandagevsSPAligner_df.to_csv('TestResultCategoriesBandageSPAligner.tsv',sep='\t')

## This section compares the values from GraphAligner and Bandage

In [31]:
master_df['Start_GraphAligner']=master_df['Start_GraphAligner'].astype(str)
master_df['End_GraphAligner']=master_df['End_GraphAligner'].astype(str)
master_df['Path_GraphAligner']=master_df['Path_GraphAligner'].astype(str)

In [32]:
BandagevsGraphAligner_df=master_df[['Query','Path_Bandage','Start_Bandage','End_Bandage','Extracted_Path',
                                'Start_GraphAligner','End_GraphAligner','Path_GraphAligner']]

In [33]:
BandagevsGraphAligner_df.dtypes


Query                 object
Path_Bandage          object
Start_Bandage         object
End_Bandage           object
Extracted_Path        object
Start_GraphAligner    object
End_GraphAligner      object
Path_GraphAligner     object
dtype: object

In [34]:
def compare_values_Bandage_GraphAligner(row):
    if row['Start_Bandage'] == row['Start_GraphAligner'] and row['End_Bandage'] == row['End_GraphAligner'] and row['Extracted_Path'] == row['Path_GraphAligner']:
        return 'Full'
    elif row['Extracted_Path'] == row['Path_GraphAligner']  and row['Start_Bandage'] != row['Start_GraphAligner'] and row['End_Bandage'] != row['End_GraphAligner'] :
        return 'MatchOnPath'
    elif row['Extracted_Path'] == row['Path_GraphAligner'] and row['Start_Bandage'] == row['Start_GraphAligner'] and row['End_Bandage'] != row['End_GraphAligner'] :
        return 'MatchOnPathAndStart'
    elif row['Extracted_Path'] == row['Path_GraphAligner'] and row['Start_Bandage'] != row['Start_GraphAligner'] and row['End_Bandage'] == row['End_GraphAligner'] :
        return 'MatchOnPathAndEnd'
    else:
        return 'Different'

In [35]:
BandagevsGraphAligner_df['BandageVSGraphAligner'] = BandagevsGraphAligner_df.apply(compare_values_Bandage_GraphAligner, axis=1)

/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_2076/607607642.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BandagevsGraphAligner_df['BandageVSGraphAligner'] = BandagevsGraphAligner_df.apply(compare_values_Bandage_GraphAligner, axis=1)


In [36]:
BandagevsGraphAligner_df.to_csv('rESULT44.tsv',sep='\t')

In [42]:
output_list1 = []

for index, row in BandagevsGraphAligner_df.iterrows():
    path_values = str(row['Extracted_Path']).split(',')
    start_values = str(row['Start_Bandage']).split(',')
    end_values = str(row['End_Bandage']).split(',')
    
    path_value = str(row['Path_GraphAligner']).split(',')
    start_value= str(row['Start_GraphAligner']).split(',')
    end_value= str(row['End_GraphAligner']).split(',')
    
    if path_value in path_values and start_value in start_values and end_value in end_values:
        output_list1.append('SinglePathFull')
    elif path_value in path_values and start_value in start_values and end_value not in end_values:
        output_list1.append('SinglePathStartMatch')
    elif path_value in path_values and start_value not in start_values and end_value  in end_values:
        output_list1.append('SinglePathEndMatch')
    elif path_value in path_values and start_value not in start_values and end_value not in end_values:
        output_list1.append('SinglePathMatch')
    else:
         output_list1.append('Different')
    

BandagevsGraphAligner_df['ResultsBandageVSGraphAligner'] = output_list1

/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_2076/2602096735.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BandagevsGraphAligner_df['ResultsBandageVSGraphAligner'] = output_list1


In [43]:
BandagevsGraphAligner_df.to_csv('rESULT445.tsv',sep='\t')

In [44]:
def determine_final_result_1(row):
    if row['BandageVSGraphAligner'] == 'Full':
        return 'Full'
    elif row['BandageVSGraphAligner'] == 'MatchOnPath':
        return 'MatchOnPath'
    else:
        return row['ResultsBandageVSGraphAligner']

In [45]:
BandagevsGraphAligner_df['FinalResultBandageVSGraphAligner'] = BandagevsGraphAligner_df.apply(determine_final_result_1, axis=1)

/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_2076/2292256490.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BandagevsGraphAligner_df['FinalResultBandageVSGraphAligner'] = BandagevsGraphAligner_df.apply(determine_final_result_1, axis=1)


In [46]:
BandagevsGraphAligner_df.to_csv('TestResultCategoriesBandageGraphAligner.tsv',sep='\t')

## This section compares the results from GraphAligner and SPAligner

In [53]:
SPAlignervsGraphAligner_df=master_df[['Query','Start_SPAligner','End_SPAligner','Path_SPAligner',
                                'Start_GraphAligner','End_GraphAligner','Path_GraphAligner','Start_SPAligner_GraphAligner']]

In [54]:
SPAlignervsGraphAligner_df.dtypes

Query                           object
Start_SPAligner                 object
End_SPAligner                   object
Path_SPAligner                  object
Start_GraphAligner              object
End_GraphAligner                object
Path_GraphAligner               object
Start_SPAligner_GraphAligner    object
dtype: object

In [55]:
def compare_values_GraphAligner_SPAligner(row):
    if row['Start_SPAligner_GraphAligner'] == row['Start_GraphAligner'] and row['End_SPAligner'] == row['End_GraphAligner'] and row['Path_SPAligner'] == row['Path_GraphAligner']:
        return 'Full'
    elif row['Path_SPAligner'] == row['Path_GraphAligner']and row['Start_SPAligner_GraphAligner'] != row['Start_GraphAligner']and row['End_SPAligner'] == row['End_GraphAligner'] :
        return 'MatchOnPathAndEnd'
    elif row['Path_SPAligner'] == row['Path_GraphAligner'] and row['Start_SPAligner_GraphAligner'] == row['Start_GraphAligner'] and row['End_SPAligner'] != row['End_GraphAligner'] :
        return 'MatchOnPathAndStart'
    elif row['Path_SPAligner'] == row['Path_GraphAligner'] and row['Start_SPAligner_GraphAligner'] != row['Start_GraphAligner'] and row['End_SPAligner'] != row['End_GraphAligner'] :
        return 'MatchOnPath'
    else:
        return 'Different'

In [56]:
SPAlignervsGraphAligner_df['SPAlignerVSGraphAligner'] = SPAlignervsGraphAligner_df.apply(compare_values_GraphAligner_SPAligner, axis=1)

/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_1851/33428813.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SPAlignervsGraphAligner_df['SPAlignerVSGraphAligner'] = SPAlignervsGraphAligner_df.apply(compare_values_GraphAligner_SPAligner, axis=1)


In [57]:
SPAlignervsGraphAligner_df.to_csv('TestResultCategoriesGraphAlignerSPAligner.tsv',sep='\t')

In [58]:


# Assuming your three data frames are named df1, df2, and df3
# Merge df1 and df2 based on the common column "Query"
merged_df_cat = pd.merge(BandagevsSPAligner_df, BandagevsGraphAligner_df, on='Query', how='inner')

# Merge the result with df3 based on the common column "Query"
final_df = pd.merge(merged_df_cat, SPAlignervsGraphAligner_df, on='Query', how='inner')


In [59]:
final_df.columns


Index(['Query', 'Path_Bandage_x', 'Start_Bandage_x', 'End_Bandage_x',
       'Extracted_Path_x', 'Start_SPAligner_x', 'End_SPAligner_x',
       'Path_SPAligner_x', 'Start_SPAligner_Bandage', 'BandageVSSPAligner',
       'ResultsBandageVSSPAligner', 'FinalResultBandageVSSPAligner',
       'Path_Bandage_y', 'Start_Bandage_y', 'End_Bandage_y',
       'Extracted_Path_y', 'Start_GraphAligner_x', 'End_GraphAligner_x',
       'Path_GraphAligner_x', 'BandageVSGraphAligner',
       'ResultsBandageVSGraphAligner', 'FinalResultBandageVSGraphAligner',
       'Start_SPAligner_y', 'End_SPAligner_y', 'Path_SPAligner_y',
       'Start_GraphAligner_y', 'End_GraphAligner_y', 'Path_GraphAligner_y',
       'Start_SPAligner_GraphAligner', 'SPAlignerVSGraphAligner'],
      dtype='object')

In [60]:
columns_to_drop = [ 'BandageVSSPAligner', 'ResultsBandageVSSPAligner','BandageVSGraphAligner',
       'ResultsBandageVSGraphAligner']

In [61]:
final_df = final_df.drop(columns=columns_to_drop)

In [62]:
final_df.to_csv('Final_df_categrories.tsv',sep='\t')